In [45]:
from postgres import PostGresClient
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [3]:
#loading in data
wines = PostGresClient().load_table('v_wine_eda')

c:\Anaconda\envs\wine_recommend\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
wines.head()

,wine_id,wine_name,wine_type,region,country,wine_style,winery_name,avg_vintage_price
0,51644,Evo,Red Wine,Ribera del Queiles,Spain,Spanish Red,Guelbenzu,38.105
1,1128181,Savigny-lès-Beaune 1er Cru 'La Dominode',Red Wine,Savigny-lès-Beaune La Dominode,France,Burgundy Côte de Beaune Red,Domaine Bruno Clair,105.225
2,6417379,Savigny-Les-Beaune 'Sous la Cabotte',Red Wine,Savigny-lès-Beaune,France,Burgundy Côte de Beaune Red,Jean Féry & Fils,49.990
3,73249,Savigny-Lès-Beaune Premier Cru Les Lavières,Red Wine,Savigny-lès-Beaune 1er Cru,France,Burgundy Côte de Beaune Red,Bouchard Père & Fils,69.375
4,1128885,Savigny-lès-Beaune 1er Cru 'Les Haut Jarrons',Red Wine,Savigny-lès-Beaune Les Haut Jarrons,France,Burgundy Côte de Beaune Red,Benjamin Leroux,51.990


In [5]:
#getting number of wines for each country
country_totals = wines.country.value_counts()

In [6]:
fig = px.choropleth(country_totals,
    locations = country_totals.index, 
    locationmode='country names', 
    color = 'country', 
    color_continuous_scale = 'Agsunset', 
    scope='world',
    labels={'country': '# Wines'}
)

fig.update_layout(
    paper_bgcolor="white",
    title = 'Total Number of Wines Produced by Country',
    margin=dict(l=0, r=0, t=50, b=10)
)
fig.show()

In [41]:
wt_counts = wines.wine_type.value_counts()
fig = px.bar(
    wt_counts,
    x = wt_counts.index,
    y = wt_counts,
    color = wt_counts.index,
    color_discrete_sequence= px.colors.qualitative.Antique,
    labels = {
        'index': 'Wine Type',
        'y' : '# of Wines'
    }
)
fig.update_layout(
    paper_bgcolor="white",
    title = 'Total Number of Wines Produced by Wine Type',
    margin=dict(l=0, r=0, t=50, b=10)
)
fig.show()

In [81]:
wt_flavorfreq = PostGresClient().load_table('v_wine_type_flavor_freq')
sorted_wtff = wt_flavorfreq.sort_values(['wine_type_name', 'cnt'], ascending=[True, False])
sorted_wtff.head()

c:\Anaconda\envs\wine_recommend\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,wine_type_name,keyword_name,cnt
191,Dessert Wine,honey,292
9,Dessert Wine,apricot,267
271,Dessert Wine,peach,251
7,Dessert Wine,apple,248
95,Dessert Wine,citrus,241


In [97]:
fig = make_subplots(rows=2, cols=3, subplot_titles=sorted_wtff.wine_type_name.unique())
for i in range(6):
    #print(i//3 + 1, '||', i%3+1)
    wt = sorted_wtff.wine_type_name.unique()[i]
    sorted_group = sorted_wtff[sorted_wtff.wine_type_name == wt].iloc[:10,:]
    fig.add_trace(
            go.Bar(
                x = sorted_group['keyword_name'],
                y = sorted_group['cnt']
            ), 
            row=i//3 + 1, 
            col=i%3+1
    )

fig.update_layout(
    showlegend = False
)
fig.show()